<a href="https://colab.research.google.com/github/lups0208/datasciencecoursera/blob/master/8_webservices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desglose

In [ ]:
import requests
import pandas as pd

Input data

In [ ]:
base_url = 'https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA'
zona = 'MONTERREY'
año_i = '2021'
mes_i = '02'
dia_i = '14'
año_f = '2021'
mes_f = '02'
dia_f = '20'

Hacemos una concatenación de los valores definidos previamente en una variable llamada ```peticion``` y se la damos como argumento a la función ```requests.get```

In [ ]:
peticion = base_url + '/' + zona + '/' + año_i + '/' + mes_i + '/' + dia_i + '/' + año_f + '/' + mes_f + '/' + dia_f + '/JSON'
response = requests.get(peticion)
response

<Response [200]>

200 simboliza que la petición tiene un status "OK", por lo que fue procesada correctamente 

In [ ]:
type(response)

requests.models.Response

Método para obtener un archivo JSON con la respuesta de nuestra petición. Observamos que es una estructura de diccionarios

In [ ]:
x = response.json()
x

Inspeccionamos los indices de este diccionario

In [ ]:
x.keys()

dict_keys(['status', 'nombre', 'proceso', 'sistema', 'area', 'Resultados'])

Los valores que nos interesan estan dentro del índice ```Resultados```

In [ ]:
type(x['Resultados'])

list

El cuál es una lista de un solo elemento

In [ ]:
len(x['Resultados'])

1

In [ ]:
type(x['Resultados'][0])

dict

El primer y único elemento es un diccionario

Guardamos en una variable llamada ```results``` este único diccionario


In [ ]:
results = x['Resultados'][0]
results.keys()

dict_keys(['zona_carga', 'Valores'])

In [ ]:
results['Valores']

Vemos que tiene dos índices este diccionario

In [ ]:
pml_df = pd.DataFrame(results['Valores'])
pml_df

,fecha,hora,pz,pz_ene,pz_per,pz_cng
0,2021-02-14,1,2282.35,2538.18,-255.83,0
1,2021-02-14,2,2082.26,2320.76,-238.5,0
2,2021-02-14,3,1827.4,2034.81,-207.41,0
3,2021-02-14,4,1765.2,1955.95,-190.74,0
4,2021-02-14,5,1715.73,1911.93,-196.19,0
...,...,...,...,...,...,...
163,2021-02-20,20,1573.97,1819.95,-164.13,-81.85
164,2021-02-20,21,1465.4,1666.44,-151,-50.04
165,2021-02-20,22,1385.55,1571.77,-152.26,-33.97
166,2021-02-20,23,1300.52,1528.74,-151.08,-77.14


El índice que nos interesa es el de ```Valores```.

Observamos el tipo de variable que tenemos en cada columna

In [ ]:
type(pml_df['fecha'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
type(pml_df['pz_ene'][0])

numpy.float64

In [ ]:
type(pml_df['hora'][0])

str

Damos formato a nuestros datos

In [ ]:
pml_df['fecha']     = pd.to_datetime(pml_df['fecha'])
pml_df['pz']        = pml_df['pz'].astype(float)
pml_df['pz_ene']    = pml_df['pz_ene'].astype(float)
pml_df['pz_cng']    = pml_df['pz_cng'].astype(float)
pml_df['periodo']   = pml_df['fecha'].astype(str) + ' : H' + pml_df['hora'].astype(str)
pml_df

,fecha,hora,pz,pz_ene,pz_per,pz_cng,periodo
0,2021-02-14,1,2282.35,2538.18,-255.83,0.00,2021-02-14 : H1
1,2021-02-14,2,2082.26,2320.76,-238.5,0.00,2021-02-14 : H2
2,2021-02-14,3,1827.40,2034.81,-207.41,0.00,2021-02-14 : H3
3,2021-02-14,4,1765.20,1955.95,-190.74,0.00,2021-02-14 : H4
4,2021-02-14,5,1715.73,1911.93,-196.19,0.00,2021-02-14 : H5
...,...,...,...,...,...,...,...
163,2021-02-20,20,1573.97,1819.95,-164.13,-81.85,2021-02-20 : H20
164,2021-02-20,21,1465.40,1666.44,-151,-50.04,2021-02-20 : H21
165,2021-02-20,22,1385.55,1571.77,-152.26,-33.97,2021-02-20 : H22
166,2021-02-20,23,1300.52,1528.74,-151.08,-77.14,2021-02-20 : H23


Finalmente podemos gráficar la información de este dataframe con ```plotly.express```

In [ ]:
import plotly.express as px
fig = px.line(pml_df, x='periodo',y='pz', title=zona)
fig.show()

In [ ]:
import datetime
fecha = datetime.datetime.now()
fecha

datetime.datetime(2022, 1, 23, 17, 11, 52, 575746)

# Función para extrar datos MDA y MTR

Definimos una función para automatizar la petición de información de PND SIN y poder comparar precios entre MDA y MTR. La idea es iterar sobre la lista ```mercados = ['MDA,'MTR]```, es decir 2 iteraciones, en donde en la primera iteración realiza la petición del MDA y la segunda iteración realiza la petición del MTR. Agregamos una columna para reconocer el mercado asociado en cada petición. Finalmente, en cada iteración, el dataframe obtenido se concatena al dataframe vacío ```plot_df```. 

In [ ]:
def zonaPML(zona,año_i,mes_i,dia_i,año_f,mes_f,dia_f):
  #Informacion requerida
  base_url = 'https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/'
  mercados = ['MDA','MTR']
  plot_df = pd.DataFrame()

  for mercado in mercados:
    #Peticion
    peticion = base_url + mercado + '/' + zona + '/' + año_i + '/' + mes_i + '/' + dia_i + '/' + año_f + '/' + mes_f + '/' + dia_f + '/JSON'
    response = requests.get(peticion)
    x = response.json()
    results = x['Resultados'][0]
    pml_zone = results['Valores']
    mem_df = pd.DataFrame(pml_zone)

    #Formato de Datos
    mem_df['fecha'] = pd.to_datetime(mem_df['fecha'])
    mem_df['pz']    = mem_df['pz'].astype(float)
    mem_df['pz_ene']    = mem_df['pz_ene'].astype(float)
    mem_df['pz_per']    = mem_df['pz_per'].astype(float)
    mem_df['pz_cng']    = mem_df['pz_cng'].astype(float)
    mem_df['periodo']   = mem_df['fecha'].astype(str) + ' : H' + mem_df['hora'].astype(str)

    #Renombramos columnas 
    mem_df.rename(columns={'pz':'Precio Zonal (MXN/MWh)'}, inplace=True)

    #creamos una nueva columna especificando el mercado 
    if mercado == 'MDA':
      mem_df['mercado'] = 'MDA'
    else:
      mem_df['mercado'] = 'MTR'

    #concatenamos el df obtenido
    plot_df = pd.concat([plot_df,mem_df], ignore_index=True)
    print(mercado + ': Peticion concretada')

  #graficamos  plot_df una vez hayamos terminado el loop
  import plotly.express as px
  fig = px.line(plot_df, x='periodo', y='Precio Zonal (MXN/MWh)', color = 'mercado', title=zona)
  fig.show()

Monterrey 2020-02-01 al 2020-02-07

In [ ]:
zonaPML('HERMOSILLO','2018','02','01','2018','02','07')

MDA: Peticion concretada
MTR: Peticion concretada
